# [STARTER] Exercise - Build a Web-Aware Agent with Search and Knowledge Comparison

In this exercise, you'll build an agent that can search the web for current information and compare
it with its internal knowledge. This demonstrates how to enhance an LLM's capabilities with real-time
web data and how to critically analyze differences between sources.


## Challenge

Your task is to create an agent that can:

- Implement web search functionality using Tavily API
- Parse and process search results effectively
- Handle different types of queries (news, facts, events)
- Extract relevant information from search results

## Setup
First, let's import the necessary libraries:

In [1]:
import os
from datetime import datetime
from typing import List, Dict
from dotenv import load_dotenv
from tavily import TavilyClient

from lib.agents import Agent
from lib.messages import BaseMessage
from lib.tooling import tool

In [2]:
load_dotenv()

True

## Play with Tavily

In [3]:
api_key = os.getenv("TAVILY_API_KEY")
client = TavilyClient(api_key=api_key)

In [4]:
# [TODO] Define a query and run
query = "What are the latest advancements in AI as of Dec 2025?"
result = client.search(query)

In [5]:
result

{'query': 'What are the latest advancements in AI as of Dec 2025?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.diagnosticimaging.com/view/advances-in-ai-december-2025',
   'title': 'Advances in AI — December 2025 - Diagnostic Imaging',
   'content': "Diagnostic Imaging's Advances in AI monthly roundup is your chance to catch up on the latest AI news, AI-powered imaging modalities and",
   'score': 0.9998623,
   'raw_content': None},
  {'url': 'https://intuitionlabs.ai/articles/latest-ai-research-trends-2025',
   'title': 'Latest AI Research (Dec 2025): GPT-5, Agents & Trends | IntuitionLabs',
   'content': 'Recent AI research (late 2025) reflects rapid advances in model capabilities, infrastructure, and application breadth.',
   'score': 0.9996699,
   'raw_content': None},
  {'url': 'https://www.riskinfo.ai/post/ai-insights-key-global-developments-in-december-2025',
   'title': 'AI Insights: Key Global Developments in December 2025 -

## Define Web Search tool

In [9]:
# [TODO] Define the search tool
@tool
def web_search(query: str, search_depth: str = "advanced") -> Dict:
    """Perform a web search using the Tavily API.
    args:
        query (str): The search query.
        search_depth (str): The depth of the search, either 'basic' or 'advanced (default)'.
    returns:
        Dict: The search results from the Tavily API.
    """
    
    api_key = os.getenv("TAVILY_API_KEY")
    client = TavilyClient(api_key=api_key)
    search_results = client.search(
        query=query,
        search_depth = search_depth,
        include_answer=True,
        include_raw_content=False,
        include_images=False,
        )
    
    formatted_results = {
        "answer": search_results.get("answer", ""),
        "sources": search_results.get("sources", []),
        "results": search_results.get("results", []),
        "search_metadata": {
            "timestamp": datetime.now().isoformat(),
            "query": query,
        }

    }
    return formatted_results

In [10]:
tools = [web_search]

## Define Agents

The first agent should not use tools, just its own knowledge. The second one should have web search tool enabled.

In [11]:
# [TODO] Agent with no tools
simple_agent = Agent(
    model_name="gpt-4o-mini",
    instructions="You are a helpful AI assistant",
)

In [12]:
# [TODO] Agent with web search tool
web_agent = Agent(
    model_name="gpt-4o-mini",
    instructions=(
        "You are a helpful AI assistant that can perform web searches updated information to answer user queries."
        "For each user query, you will use the web search tool using Tavily's AI-optimized search to find relevant information and provide a comprehensive response."
        "Always cite your sources from the search results in your final answer."
        "Be careful to dates and time-sensitive information."),
    tools=tools
)

In [13]:
def print_messages(messages: List[BaseMessage]):
    for m in messages:
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

## Run your Agents

Run the Agents and compare them. The following queries are just for reference. Change them as you want.

**Simple Agent**

**Note**: This example relies on the date being recent enough that the answer will not be in the model's training data. Try with other current events/dates if needed to get similar results.

In [14]:
run1 = simple_agent.invoke(
    query="Who won the 2025 Oscar for International Movie?", 
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are a helpful AI assistant, tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = I'm sorry, but I don't have information on events that occurred after October 2023, including the winners of the 2025 Oscars. You may want to check the latest news or the official Oscars website for the most up-to-date information., tool_calls = None)


In [15]:
print(run1.get_final_state()["messages"][-1].content)

I'm sorry, but I don't have information on events that occurred after October 2023, including the winners of the 2025 Oscars. You may want to check the latest news or the official Oscars website for the most up-to-date information.


In [16]:
run2 = simple_agent.invoke(
    query="What are the most recent developments in AI technology?", 
)
print("\nMessages from run 2:")
messages = run2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = You are a helpful AI assistant, tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = I'm sorry, but I don't have information on events that occurred after October 2023, including the winners of the 2025 Oscars. You may want to check the latest news or the official Oscars website for the most up-to-date information., tool_calls = None)
 -> (role = user, content = What are the most recent developments in AI technology?, tool_calls = None)
 -> (role = assistant, content = As of my last update in October 2023, several significant developments in AI technology were making headlines:

1. **Generative AI Advancements**: Models like GPT-4 and other generative AI systems were becoming 

In [17]:
print(run2.get_final_state()["messages"][-1].content)

As of my last update in October 2023, several significant developments in AI technology were making headlines:

1. **Generative AI Advancements**: Models like GPT-4 and other generative AI systems were becoming more sophisticated in natural language processing, image generation, and even video synthesis. These models were being fine-tuned for various applications, including creative writing, coding assistance, and content creation.

2. **AI in Healthcare**: AI technologies were increasingly being used in healthcare for diagnostics, treatment planning, and personalized medicine. Machine learning algorithms were being developed to analyze medical imaging and predict patient outcomes more accurately.

3. **Ethical AI and Regulation**: There was a growing focus on the ethical implications of AI, with discussions and proposals around regulations to ensure responsible AI development and deployment. This included transparency, bias mitigation, and accountability in AI systems.

4. **AI in Aut

**Web Agent**

In [18]:
run1 = web_agent.invoke(
    query="Who won the 2025 Oscar for International Movie?", 
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are a helpful AI assistant that can perform web searches updated information to answer user queries.For each user query, you will use the web search tool using Tavily's AI-optimized search to find relevant information and provide a comprehensive response.Always cite your sources from the search results in your final answer.Be careful to dates and time-sensitive information., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageToolCall(id='call_6yYx0sETZlYLKliUgardtB52', function=Function(arguments='{"query":"2025 Oscar for International Movie 

In [19]:
print(run1.get_final_state()["messages"][-1].content)

The 2025 Oscar for Best International Feature was awarded to Brazil's film **"I'm Still Here."** The film received three nominations, including Best Picture and Best Actress for Fernanda Torres. This recognition highlights the film's significant impact and popularity at the Oscars, showcasing a strong narrative that resonated with audiences and critics alike.

For more details, you can read about it on the official NPR article [here](https://www.npr.org/2025/03/02/nx-s1-5315313/oscars-2025-im-still-here-brazil-best-international-feature) or on People.com [here](https://people.com/oscars-2025-im-still-here-brazil-best-international-feature-11684447).


In [20]:
run2 = web_agent.invoke(
    query="What are the most recent developments in AI technology?", 
)
print("\nMessages from run 2:")
messages = run2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = You are a helpful AI assistant that can perform web searches updated information to answer user queries.For each user query, you will use the web search tool using Tavily's AI-optimized search to find relevant information and provide a comprehensive response.Always cite your sources from the search results in your final answer.Be careful to dates and time-sensitive information., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageToolCall(id='call_6yYx0sETZlYLKliUgardtB52', function=Function(arguments='{"query":"2025 Oscar for International Movie 

In [21]:
print(run2.get_final_state()["messages"][-1].content)

As of 2025, there have been several significant developments in AI technology:

1. **Advanced AI Models**: Major breakthroughs have been achieved with AI models like **Gemini 3** and **Gemma 3**, improving reasoning capabilities and overall performance in various applications. These models are part of a broader trend toward more sophisticated AI systems that can understand and process information more effectively.

2. **Tesla's Humanoid Robot**: Tesla introduced the **Optimus** humanoid robot, showcasing their advancements in AI-driven robotics. This robot is designed to assist in various tasks, marking a significant step toward integrating AI in physical environments.

3. **Open-Source AI Growth**: There has been a notable rise in open-source AI initiatives, particularly in regions like China, which has been actively developing its AI capabilities. This trend is democratizing access to powerful AI tools and fostering innovation across different sectors.

4. **Human-Machine Relationshi

## Advanced

You can modify `agents.py` to include: 
- a comparison field in the state schema
- a web search step
- a comparison step in the workflow